# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 18 2022 2:25PM,242125,10,8291627,"ClearSpec, LLC",Released
1,Jul 18 2022 2:24PM,242123,19,ADV80004258,"AdvaGen Pharma, Ltd",Released
2,Jul 18 2022 2:24PM,242123,19,ADV80004259,"AdvaGen Pharma, Ltd",Released
3,Jul 18 2022 2:24PM,242123,19,ADV80004260,"AdvaGen Pharma, Ltd",Released
4,Jul 18 2022 2:24PM,242123,19,ADV80004261,"AdvaGen Pharma, Ltd",Released
5,Jul 18 2022 2:24PM,242123,19,ADV80004262,"AdvaGen Pharma, Ltd",Released
6,Jul 18 2022 2:24PM,242123,19,ADV80004263,"AdvaGen Pharma, Ltd",Released
7,Jul 18 2022 2:24PM,242123,19,ADV80004264,"AdvaGen Pharma, Ltd",Released
8,Jul 18 2022 2:24PM,242123,19,ADV80004265,"AdvaGen Pharma, Ltd",Released
9,Jul 18 2022 2:24PM,242123,19,ADV80004266,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,242121,Released,21
50,242122,Released,1
51,242123,Released,23
52,242124,Released,1
53,242125,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242121,NaN,NaN,21.0
242122,NaN,NaN,1.0
242123,NaN,NaN,23.0
242124,NaN,NaN,1.0
242125,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,0.0,0.0,12.0
241972,0.0,9.0,5.0
241980,0.0,0.0,31.0
242005,0.0,0.0,3.0
242007,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,0,0,12
241972,0,9,5
241980,0,0,31
242005,0,0,3
242007,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,0,0,12
1,241972,0,9,5
2,241980,0,0,31
3,242005,0,0,3
4,242007,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,,,12
1,241972,,9,5
2,241980,,,31
3,242005,,,3
4,242007,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 18 2022 2:25PM,242125,10,"ClearSpec, LLC"
1,Jul 18 2022 2:24PM,242123,19,"AdvaGen Pharma, Ltd"
24,Jul 18 2022 2:23PM,242124,21,"NBTY Global, Inc."
25,Jul 18 2022 2:21PM,242122,21,"NBTY Global, Inc."
26,Jul 18 2022 2:14PM,242121,20,"Exact-Rx, Inc."
47,Jul 18 2022 2:08PM,242120,19,ACG North America LLC
48,Jul 18 2022 2:02PM,242119,10,Bio-PRF
64,Jul 18 2022 1:59PM,242118,10,"Methapharm, Inc."
78,Jul 18 2022 1:56PM,242117,22,"NBTY Global, Inc."
79,Jul 18 2022 1:53PM,241900,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 18 2022 2:25PM,242125,10,"ClearSpec, LLC",,,1
1,Jul 18 2022 2:24PM,242123,19,"AdvaGen Pharma, Ltd",,,23
2,Jul 18 2022 2:23PM,242124,21,"NBTY Global, Inc.",,,1
3,Jul 18 2022 2:21PM,242122,21,"NBTY Global, Inc.",,,1
4,Jul 18 2022 2:14PM,242121,20,"Exact-Rx, Inc.",,,21
5,Jul 18 2022 2:08PM,242120,19,ACG North America LLC,,,1
6,Jul 18 2022 2:02PM,242119,10,Bio-PRF,,,16
7,Jul 18 2022 1:59PM,242118,10,"Methapharm, Inc.",,,14
8,Jul 18 2022 1:56PM,242117,22,"NBTY Global, Inc.",,,1
9,Jul 18 2022 1:53PM,241900,20,Alumier Labs Inc.,,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 2:25PM,242125,10,"ClearSpec, LLC",1,,
1,Jul 18 2022 2:24PM,242123,19,"AdvaGen Pharma, Ltd",23,,
2,Jul 18 2022 2:23PM,242124,21,"NBTY Global, Inc.",1,,
3,Jul 18 2022 2:21PM,242122,21,"NBTY Global, Inc.",1,,
4,Jul 18 2022 2:14PM,242121,20,"Exact-Rx, Inc.",21,,
5,Jul 18 2022 2:08PM,242120,19,ACG North America LLC,1,,
6,Jul 18 2022 2:02PM,242119,10,Bio-PRF,16,,
7,Jul 18 2022 1:59PM,242118,10,"Methapharm, Inc.",14,,
8,Jul 18 2022 1:56PM,242117,22,"NBTY Global, Inc.",1,,
9,Jul 18 2022 1:53PM,241900,20,Alumier Labs Inc.,12,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 2:25PM,242125,10,"ClearSpec, LLC",1,,
1,Jul 18 2022 2:24PM,242123,19,"AdvaGen Pharma, Ltd",23,,
2,Jul 18 2022 2:23PM,242124,21,"NBTY Global, Inc.",1,,
3,Jul 18 2022 2:21PM,242122,21,"NBTY Global, Inc.",1,,
4,Jul 18 2022 2:14PM,242121,20,"Exact-Rx, Inc.",21,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 2:25PM,242125,10,"ClearSpec, LLC",1.0,NaN,NaN
1,Jul 18 2022 2:24PM,242123,19,"AdvaGen Pharma, Ltd",23.0,NaN,NaN
2,Jul 18 2022 2:23PM,242124,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jul 18 2022 2:21PM,242122,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jul 18 2022 2:14PM,242121,20,"Exact-Rx, Inc.",21.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 2:25PM,242125,10,"ClearSpec, LLC",1.0,0.0,0.0
1,Jul 18 2022 2:24PM,242123,19,"AdvaGen Pharma, Ltd",23.0,0.0,0.0
2,Jul 18 2022 2:23PM,242124,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jul 18 2022 2:21PM,242122,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jul 18 2022 2:14PM,242121,20,"Exact-Rx, Inc.",21.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3630990,126.0,30.0,0.0
102,242008,1.0,0.0,0.0
15,484119,6.0,10.0,10.0
19,968427,27.0,0.0,0.0
20,2178357,93.0,18.0,0.0
21,3873517,14.0,2.0,0.0
22,484232,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3630990,126.0,30.0,0.0
1,102,242008,1.0,0.0,0.0
2,15,484119,6.0,10.0,10.0
3,19,968427,27.0,0.0,0.0
4,20,2178357,93.0,18.0,0.0
5,21,3873517,14.0,2.0,0.0
6,22,484232,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,126.0,30.0,0.0
1,102,1.0,0.0,0.0
2,15,6.0,10.0,10.0
3,19,27.0,0.0,0.0
4,20,93.0,18.0,0.0
5,21,14.0,2.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,126.0
1,102,Released,1.0
2,15,Released,6.0
3,19,Released,27.0
4,20,Released,93.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
Executing,30.0,0.0,10.0,0.0,18.0,2.0,0.0
Released,126.0,1.0,6.0,27.0,93.0,14.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,19,20,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
1,Executing,30.0,0.0,10.0,0.0,18.0,2.0,0.0
2,Released,126.0,1.0,6.0,27.0,93.0,14.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,19,20,21,22
0,Completed,0.0,0.0,10.0,0.0,0.0,0.0,0.0
1,Executing,30.0,0.0,10.0,0.0,18.0,2.0,0.0
2,Released,126.0,1.0,6.0,27.0,93.0,14.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()